In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('data/adult.data.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## 1. Сколько мужчин и женщин (признак sex) представлено в этом наборе данных?

In [7]:
df['sex'].value_counts()

Male      21790
Female    10771
Name: sex, dtype: int64

## 2. Каков средний возраст (признак age) женщин?

In [12]:
df[df['sex'] == 'Female']['age'].mean()

36.85823043357163

## 3. Какова доля граждан Германии (признак native-country)?

In [20]:
df['native-country'].value_counts(normalize=True)['Germany']

0.004207487485028101

## 4. Каковы средние значения и среднеквадратичные отклонения возраста тех, кто получает более 50K в год (признак salary)?
## 5. ... кто получает менее 50K в год?

In [33]:
df.groupby(['salary'])['age'].agg([np.mean, np.std])

,mean,std
salary,,
<=50K,36.783738,14.020088
>50K,44.249841,10.519028


## 6. Правда ли, что люди, которые получают больше 50k, имеют как минимум высшее образование? (признак education – Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters или Doctorate)

In [34]:
pd.crosstab(df['salary'], df['education'])

education,10th,11th,12th,1st-4th,5th-6th,7th-8th,9th,Assoc-acdm,Assoc-voc,Bachelors,Doctorate,HS-grad,Masters,Preschool,Prof-school,Some-college
salary,,,,,,,,,,,,,,,,
<=50K,871,1115,400,162,317,606,487,802,1021,3134,107,8826,764,51,153,5904
>50K,62,60,33,6,16,40,27,265,361,2221,306,1675,959,0,423,1387


Ответ: нет

## 7. Выведите статистику возраста для каждой расы (признак race) и каждого пола. Используйте groupby и describe. Найдите таким образом максимальный возраст мужчин расы Amer-Indian-Eskimo.

**Female statistic:**

In [37]:
df[df['sex'] == 'Female'].groupby(['race'])['age'].describe(percentiles = [])

,count,mean,std,min,50%,max
race,,,,,,
Amer-Indian-Eskimo,119.0,37.117647,13.114991,17.0,36.0,80.0
Asian-Pac-Islander,346.0,35.089595,12.300845,17.0,33.0,75.0
Black,1555.0,37.854019,12.637197,17.0,37.0,90.0
Other,109.0,31.678899,11.631599,17.0,29.0,74.0
White,8642.0,36.811618,14.329093,17.0,35.0,90.0


**Male statistic:**

In [38]:
df[df['sex'] == 'Male'].groupby(['race'])['age'].describe(percentiles = [])

,count,mean,std,min,50%,max
race,,,,,,
Amer-Indian-Eskimo,192.0,37.208333,12.049563,17.0,35.0,82.0
Asian-Pac-Islander,693.0,39.073593,12.883944,18.0,37.0,90.0
Black,1569.0,37.682600,12.882612,17.0,36.0,90.0
Other,162.0,34.654321,11.355531,17.0,32.0,77.0
White,19174.0,39.652498,13.436029,17.0,38.0,90.0


## 8. Среди кого больше доля зарабатывающих много (>50K): среди женатых или холостых мужчин (признак marital-status)? Женатыми считаем тех, у кого marital-status начинается с Married (Married-civ-spouse, Married-spouse-absent или Married-AF-spouse), остальных считаем холостыми.

In [42]:
df2 = df[df['sex'] == 'Male']
pd.crosstab(df2['marital-status'], df2['salary'])

salary,<=50K,>50K
marital-status,,
Divorced,1487,284
Married-AF-spouse,5,4
Married-civ-spouse,7381,5938
Married-spouse-absent,190,23
Never-married,5591,325
Separated,345,49
Widowed,129,39


## 9. Какое максимальное число часов человек работает в неделю (признак hours-per-week)? Сколько людей работают такое количество часов и каков среди них процент зарабатывающих много?

In [7]:
max_hours = df['hours-per-week'].max()
print("Макс. часов/неделю: ",max_hours)

max_people = df[df['hours-per-week'] == max_hours].shape[0]
print("Люди, работающие макс. час/неделю: ", max_people)

rich = float(df[(df['hours-per-week'] == max_hours)
                 & (df['salary'] == '>50K')].shape[0]) / max_people
print("Из них богатых {0}%".format(int(100 * rich)))

Макс. часов/неделю:  99
Люди, работающие макс. час/неделю:  85
Из них богатых 29%


## 10. Посчитайте среднее время работы (hours-per-week) зарабатывающих мало и много (salary) для каждой страны (native-country).


In [79]:
countries = set(df['native-country'])
for c in countries:
    big = df[(df['salary'] == '>50K') & (df['native-country'] == c)]['hours-per-week'].mean()
    small = df[(df['salary'] == '<=50K') & (df['native-country'] == c)]['hours-per-week'].mean()
    print(c, "|", "high salary:", big, 'h/week', "| low salary:", small, 'h/week')
    print('---')

South | high salary: 51.4375 h/week | low salary: 40.15625 h/week
---
Canada | high salary: 45.64102564102564 h/week | low salary: 37.91463414634146 h/week
---
Haiti | high salary: 42.75 h/week | low salary: 36.325 h/week
---
United-States | high salary: 45.50536884674383 h/week | low salary: 38.79912723305605 h/week
---
Puerto-Rico | high salary: 39.416666666666664 h/week | low salary: 38.470588235294116 h/week
---
El-Salvador | high salary: 45.0 h/week | low salary: 36.03092783505155 h/week
---
Yugoslavia | high salary: 49.5 h/week | low salary: 41.6 h/week
---
Hong | high salary: 45.0 h/week | low salary: 39.142857142857146 h/week
---
Japan | high salary: 47.958333333333336 h/week | low salary: 41.0 h/week
---
Mexico | high salary: 46.57575757575758 h/week | low salary: 40.00327868852459 h/week
---
Philippines | high salary: 43.032786885245905 h/week | low salary: 38.065693430656935 h/week
---
Germany | high salary: 44.97727272727273 h/week | low salary: 39.13978494623656 h/week
---